# Transformation of scrapped data

### Sommaire

1- Loading data

2- Removing for duplicates

3- Making a Join on comments with posts

4- Add a feature sentiment analysis

### 1- Loading data

In [1]:
import glob
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# getting csv files from the folder
path = "../scrapping/csv_exports/comments"

# read all the files with extension .csv
filenames = glob.glob(path + "\*.csv")
all_comments = pd.DataFrame()
# for loop to iterate all csv files
for file in filenames:
   # reading csv files
   print("\nReading file = ",file)
   all_comments = all_comments.append(pd.read_csv(file))

all_comments = all_comments.reset_index(drop=True)
all_comments


Reading file =  ../scrapping/csv_exports/comments\france_comments_pmgb4s.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_pt7d6b.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_q10xcm.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_q8ms87.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_qgu7rx.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_qq0qkz.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_qys6zk.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_r6infj.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_re7845.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_rm98pa.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_ruvgu9.csv

Reading file =  ../scrapping/csv_exports/comments\france_comments_s175tv.csv

Reading file =  ../scrapping/csv_exports/comments\france_commen

,post_id,authors,commentId,text,parent_commentId,created,permalink
0,pfg0r5,_Tripe_,hb42jo8,Sur ce coup là 🤡\n\nAprès faut voir sur quelle...,t3_pfg0r5,2021-09-01 00:08:06,/r/france/comments/pfg0r5/bravo_la_france_on_b...
1,pfg0r5,TB54,hb43vqt,J'avoue que j'ai vraiment du mal à croire le r...,t3_pfg0r5,2021-09-01 00:17:45,/r/france/comments/pfg0r5/bravo_la_france_on_b...
2,pfg0r5,coelhophisis,hb44wb6,"En même temps, si j'ai un membre de ma famille...",t3_pfg0r5,2021-09-01 00:25:07,/r/france/comments/pfg0r5/bravo_la_france_on_b...
3,pfg0r5,[deleted],hb4568g,[removed],t3_pfg0r5,2021-09-01 00:27:08,/r/france/comments/pfg0r5/bravo_la_france_on_b...
4,pfg0r5,CulteDeLaRaison,hb45ugd,"Je veux bien la taille de la population, la mé...",t3_pfg0r5,2021-09-01 00:32:07,/r/france/comments/pfg0r5/bravo_la_france_on_b...
...,...,...,...,...,...,...,...
1337914,ucbp6h,OverripeMandrake,i6a2huq,"Oui, ce variant est tellement agressif que les...",t1_i69f22e,2022-04-26 17:48:15,/r/france/comments/ucbp6h/every_russian_oligar...
1337915,ucbp6h,tanaph777,i6as7nh,Qu'ils butent les oligarques potentiellemet ré...,t3_ucbp6h,2022-04-26 20:33:23,/r/france/comments/ucbp6h/every_russian_oligar...
1337916,ucbp6h,fdesouche,i6b4wbb,L’article passe très vite sur les 4 enfants…,t3_ucbp6h,2022-04-26 21:54:40,/r/france/comments/ucbp6h/every_russian_oligar...
1337917,ucbp6h,xroche,i6dd28a,_L'hypothèse de l'accident n'est cependant.pas...,t1_i69g0al,2022-04-27 08:35:05,/r/france/comments/ucbp6h/every_russian_oligar...


In [2]:
import glob

# getting csv files from the folder
path = "../scrapping/csv_exports"

# read all the files with extension .csv
filenames = glob.glob(path + "\*.csv")
print('File names:', filenames)
all_titres = pd.DataFrame()
# for loop to iterate all csv files
for file in filenames:
   # reading csv files
   print("\nReading file = ",file)
   all_titres = all_titres.append(pd.read_csv(file))

all_titres = all_titres.reset_index(drop=True)
all_titres

File names: ['../scrapping/csv_exports\\france_20210901_20211002.csv', '../scrapping/csv_exports\\france_20211002_20211031.csv', '../scrapping/csv_exports\\france_20211031_20211231.csv', '../scrapping/csv_exports\\france_20211231_20220131.csv', '../scrapping/csv_exports\\france_20220131_20220228.csv', '../scrapping/csv_exports\\france_20220228_20220331.csv', '../scrapping/csv_exports\\france_20220331_20220401.csv', '../scrapping/csv_exports\\france_20220331_20220426.csv']

Reading file =  ../scrapping/csv_exports\france_20210901_20211002.csv

Reading file =  ../scrapping/csv_exports\france_20211002_20211031.csv

Reading file =  ../scrapping/csv_exports\france_20211031_20211231.csv

Reading file =  ../scrapping/csv_exports\france_20211231_20220131.csv

Reading file =  ../scrapping/csv_exports\france_20220131_20220228.csv

Reading file =  ../scrapping/csv_exports\france_20220228_20220331.csv

Reading file =  ../scrapping/csv_exports\france_20220331_20220401.csv

Reading file =  ../scrapp

,postId,title,body,url,author,created,permalink,flair
0,pfg0r5,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
1,pfgezw,CD blue ray raye,[removed],https://www.reddit.com/r/france/comments/pfgez...,Successful_Sport_565,2021-09-01 00:27:12,/r/france/comments/pfgezw/cd_blue_ray_raye/,NaN
2,pfh52l,"Dans la Creuse, des tags anti-vaccins et antis...",NaN,https://www.liberation.fr/politique/dans-la-cr...,Tigrafr,2021-09-01 01:07:33,/r/france/comments/pfh52l/dans_la_creuse_des_t...,Paywall
3,pfh9pl,Une histoire de manipulation et de violences,[removed],https://www.reddit.com/r/france/comments/pfh9p...,temp7894561230,2021-09-01 01:14:51,/r/france/comments/pfh9pl/une_histoire_de_mani...,NaN
4,pfhakb,Pourquoi la gestion de la désinformation par R...,Suite à [ce poteau](https://www.reddit.com/r/f...,https://www.reddit.com/r/france/comments/pfhak...,dixfoisdix,2021-09-01 01:16:08,/r/france/comments/pfhakb/pourquoi_la_gestion_...,Méta
...,...,...,...,...,...,...,...,...
42266,ucb6cz,De la guerre à la drogue à la prévention des a...,NaN,/r/AddictionsFR/comments/uc5v6q/de_la_guerre_à...,unmalepourunbien,2022-04-26 14:14:47,/r/france/comments/ucb6cz/de_la_guerre_à_la_dr...,Société
42267,ucbjgz,Claude Francois - Si j'avais un marteau - 1963,NaN,https://www.youtube.com/watch?v=Y7nrcb3oGkc,not_franck_the_cook,2022-04-26 14:34:43,/r/france/comments/ucbjgz/claude_francois_si_j...,Musique
42268,ucboq7,Le cabinet de Brigitte Macron déclenche l’insp...,NaN,https://www.mediapart.fr/journal/france/260422...,ThymEtRomarin,2022-04-26 14:42:17,/r/france/comments/ucboq7/le_cabinet_de_brigit...,Paywall
42269,ucboz4,Mon dada : détourner la culture pop américaine...,NaN,https://v.redd.it/hrm5kn7l0vv81,shalli,2022-04-26 14:42:38,/r/france/comments/ucboz4/mon_dada_détourner_l...,Cinéma Séries


### 2- Removing duplicates

In [3]:
all_comments = all_comments.drop_duplicates()
all_comments.rename(columns = {'commentId':'comment_id', 'parent_commentId':'parent_comment_id'}, inplace = True)
all_comments

C:\Users\matth\AppData\Local\Temp\ipykernel_13900\289693248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_comments.rename(columns = {'commentId':'comment_id', 'parent_commentId':'parent_comment_id'}, inplace = True)


,post_id,authors,comment_id,text,parent_comment_id,created,permalink
0,pfg0r5,_Tripe_,hb42jo8,Sur ce coup là 🤡\n\nAprès faut voir sur quelle...,t3_pfg0r5,2021-09-01 00:08:06,/r/france/comments/pfg0r5/bravo_la_france_on_b...
1,pfg0r5,TB54,hb43vqt,J'avoue que j'ai vraiment du mal à croire le r...,t3_pfg0r5,2021-09-01 00:17:45,/r/france/comments/pfg0r5/bravo_la_france_on_b...
2,pfg0r5,coelhophisis,hb44wb6,"En même temps, si j'ai un membre de ma famille...",t3_pfg0r5,2021-09-01 00:25:07,/r/france/comments/pfg0r5/bravo_la_france_on_b...
3,pfg0r5,[deleted],hb4568g,[removed],t3_pfg0r5,2021-09-01 00:27:08,/r/france/comments/pfg0r5/bravo_la_france_on_b...
4,pfg0r5,CulteDeLaRaison,hb45ugd,"Je veux bien la taille de la population, la mé...",t3_pfg0r5,2021-09-01 00:32:07,/r/france/comments/pfg0r5/bravo_la_france_on_b...
...,...,...,...,...,...,...,...
1337914,ucbp6h,OverripeMandrake,i6a2huq,"Oui, ce variant est tellement agressif que les...",t1_i69f22e,2022-04-26 17:48:15,/r/france/comments/ucbp6h/every_russian_oligar...
1337915,ucbp6h,tanaph777,i6as7nh,Qu'ils butent les oligarques potentiellemet ré...,t3_ucbp6h,2022-04-26 20:33:23,/r/france/comments/ucbp6h/every_russian_oligar...
1337916,ucbp6h,fdesouche,i6b4wbb,L’article passe très vite sur les 4 enfants…,t3_ucbp6h,2022-04-26 21:54:40,/r/france/comments/ucbp6h/every_russian_oligar...
1337917,ucbp6h,xroche,i6dd28a,_L'hypothèse de l'accident n'est cependant.pas...,t1_i69g0al,2022-04-27 08:35:05,/r/france/comments/ucbp6h/every_russian_oligar...


In [4]:
all_titres = all_titres.drop_duplicates()
all_titres.rename(columns = {'postId':'post_id'}, inplace = True)
all_titres

,post_id,title,body,url,author,created,permalink,flair
0,pfg0r5,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
1,pfgezw,CD blue ray raye,[removed],https://www.reddit.com/r/france/comments/pfgez...,Successful_Sport_565,2021-09-01 00:27:12,/r/france/comments/pfgezw/cd_blue_ray_raye/,NaN
2,pfh52l,"Dans la Creuse, des tags anti-vaccins et antis...",NaN,https://www.liberation.fr/politique/dans-la-cr...,Tigrafr,2021-09-01 01:07:33,/r/france/comments/pfh52l/dans_la_creuse_des_t...,Paywall
3,pfh9pl,Une histoire de manipulation et de violences,[removed],https://www.reddit.com/r/france/comments/pfh9p...,temp7894561230,2021-09-01 01:14:51,/r/france/comments/pfh9pl/une_histoire_de_mani...,NaN
4,pfhakb,Pourquoi la gestion de la désinformation par R...,Suite à [ce poteau](https://www.reddit.com/r/f...,https://www.reddit.com/r/france/comments/pfhak...,dixfoisdix,2021-09-01 01:16:08,/r/france/comments/pfhakb/pourquoi_la_gestion_...,Méta
...,...,...,...,...,...,...,...,...
42266,ucb6cz,De la guerre à la drogue à la prévention des a...,NaN,/r/AddictionsFR/comments/uc5v6q/de_la_guerre_à...,unmalepourunbien,2022-04-26 14:14:47,/r/france/comments/ucb6cz/de_la_guerre_à_la_dr...,Société
42267,ucbjgz,Claude Francois - Si j'avais un marteau - 1963,NaN,https://www.youtube.com/watch?v=Y7nrcb3oGkc,not_franck_the_cook,2022-04-26 14:34:43,/r/france/comments/ucbjgz/claude_francois_si_j...,Musique
42268,ucboq7,Le cabinet de Brigitte Macron déclenche l’insp...,NaN,https://www.mediapart.fr/journal/france/260422...,ThymEtRomarin,2022-04-26 14:42:17,/r/france/comments/ucboq7/le_cabinet_de_brigit...,Paywall
42269,ucboz4,Mon dada : détourner la culture pop américaine...,NaN,https://v.redd.it/hrm5kn7l0vv81,shalli,2022-04-26 14:42:38,/r/france/comments/ucboz4/mon_dada_détourner_l...,Cinéma Séries


We know check if we still have duplicated on the IDs of both dataframes

In [5]:
assert len(all_comments[all_comments.duplicated(['comment_id'])]) == 0, "Meh, I found some duplicated comments IDs in the dataframe"
assert len(all_titres[all_titres.duplicated(['post_id'])]) == 0, "Meh, I found some duplicated post IDs in the dataframe"

### 3- Making a Join on comments with posts

In [6]:
merged_df = all_comments.merge(all_titres, on="post_id", how = "left")
merged_df.rename(columns = {'authors':'author_comment','author':'author_post', 'created_y':'created_post', 'permalink_x':'permalink_comment', 'permalink_y':'permalink_post'}, inplace = True)
merged_df

,post_id,author_comment,comment_id,text,parent_comment_id,created_x,permalink_comment,title,body,url,author_post,created_post,permalink_post,flair
0,pfg0r5,_Tripe_,hb42jo8,Sur ce coup là 🤡\n\nAprès faut voir sur quelle...,t3_pfg0r5,2021-09-01 00:08:06,/r/france/comments/pfg0r5/bravo_la_france_on_b...,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
1,pfg0r5,TB54,hb43vqt,J'avoue que j'ai vraiment du mal à croire le r...,t3_pfg0r5,2021-09-01 00:17:45,/r/france/comments/pfg0r5/bravo_la_france_on_b...,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
2,pfg0r5,coelhophisis,hb44wb6,"En même temps, si j'ai un membre de ma famille...",t3_pfg0r5,2021-09-01 00:25:07,/r/france/comments/pfg0r5/bravo_la_france_on_b...,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
3,pfg0r5,[deleted],hb4568g,[removed],t3_pfg0r5,2021-09-01 00:27:08,/r/france/comments/pfg0r5/bravo_la_france_on_b...,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
4,pfg0r5,CulteDeLaRaison,hb45ugd,"Je veux bien la taille de la population, la mé...",t3_pfg0r5,2021-09-01 00:32:07,/r/france/comments/pfg0r5/bravo_la_france_on_b...,"Bravo la France, on bat tous les records",NaN,https://i.redd.it/hsbrhsc0kpk71.jpg,Feisty-Ad-7455,2021-09-01 00:05:43,/r/france/comments/pfg0r5/bravo_la_france_on_b...,Société
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186265,ucbp6h,OverripeMandrake,i6a2huq,"Oui, ce variant est tellement agressif que les...",t1_i69f22e,2022-04-26 17:48:15,/r/france/comments/ucbp6h/every_russian_oligar...,Every Russian oligarch who has died since Puti...,NaN,https://www.newsweek.com/every-russian-oligarc...,Tyrajausore,2022-04-26 14:42:57,/r/france/comments/ucbp6h/every_russian_oligar...,Politique
1186266,ucbp6h,tanaph777,i6as7nh,Qu'ils butent les oligarques potentiellemet ré...,t3_ucbp6h,2022-04-26 20:33:23,/r/france/comments/ucbp6h/every_russian_oligar...,Every Russian oligarch who has died since Puti...,NaN,https://www.newsweek.com/every-russian-oligarc...,Tyrajausore,2022-04-26 14:42:57,/r/france/comments/ucbp6h/every_russian_oligar...,Politique
1186267,ucbp6h,fdesouche,i6b4wbb,L’article passe très vite sur les 4 enfants…,t3_ucbp6h,2022-04-26 21:54:40,/r/france/comments/ucbp6h/every_russian_oligar...,Every Russian oligarch who has died since Puti...,NaN,https://www.newsweek.com/every-russian-oligarc...,Tyrajausore,2022-04-26 14:42:57,/r/france/comments/ucbp6h/every_russian_oligar...,Politique
1186268,ucbp6h,xroche,i6dd28a,_L'hypothèse de l'accident n'est cependant.pas...,t1_i69g0al,2022-04-27 08:35:05,/r/france/comments/ucbp6h/every_russian_oligar...,Every Russian oligarch who has died since Puti...,NaN,https://www.newsweek.com/every-russian-oligarc...,Tyrajausore,2022-04-26 14:42:57,/r/france/comments/ucbp6h/every_russian_oligar...,Politique


### 4- Add a feature sentiment analysis

a- Construction of a NLP pipeline to clean the comments

In [32]:
import re

def nlp_pipeline(comment) -> str:
    comment = str(comment).lower()
    comment = comment.replace('\n', ' ').replace('\r', '')
    comment = ' '.join(comment.split())
    comment = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", comment)
    comment = re.sub(r"(\s\-\s|-$)", "", comment)
    comment = re.sub(r"[,\!\?\%\(\)\/\"]", "", comment)
    comment = re.sub(r"\&\S*\s", "", comment)
    comment = re.sub(r"\&", "", comment)
    comment = re.sub(r"\+", "", comment)
    comment = re.sub(r"\#", "", comment)
    comment = re.sub(r"\$", "", comment)
    comment = re.sub(r"\£", "", comment)
    comment = re.sub(r"\%", "", comment)
    comment = re.sub(r"\:", "", comment)
    comment = re.sub(r"\@", "", comment)
    comment = re.sub(r"\-", "", comment)

    return comment

In [24]:
test_comment = merged_df['text'][5]
print('## Original comment ##')
print(test_comment)
print('## Processed comment ##')
print(nlp_pipeline(test_comment))

## Original comment ##
Mouais, faut voir la façon dont ils ont formulé ça dans les différentes langues.

Ca peut tout et rien dire "supportive"
## Processed comment ##
mouais faut voir la façon dont ils ont formulé ça dans les différentes langues. ca peut tout et rien dire supportive


Setting stopwords

In [27]:
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS

stop_words = set(STOP_WORDS)
deselect_stop_words = ['ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

Applying pipeline and removing stopwords from our dataframe

In [34]:
#1 min 30
merged_df['text_processed'] = merged_df['text'].apply(nlp_pipeline)

#12.6 sec
merged_df['text_processed'] = merged_df['text_processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
merged_df['text_processed'][5]

'mouais faut voir formulé langues. ca rien supportive'

In [35]:
print('## Original comment ##')
print(merged_df['text'][5])
print('## Processed comment ##')
print(merged_df['text_processed'][5])

## Original comment ##
Mouais, faut voir la façon dont ils ont formulé ça dans les différentes langues.

Ca peut tout et rien dire "supportive"
## Processed comment ##
mouais faut voir formulé langues. ca rien supportive


b- Using the commentBlob library to get sentiment analysis